# Sentiments towards COVID-19 Vaccine in South Africa

## Background
What were we trying to achieve? What are the key results/insights did we obtain? How did we get to these insights? What data was used and how was it processed? 

In [4]:
#import keras

In [2]:
# Importing useful packages
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from stop_words import get_stop_words
import re
from english_words import english_words_set
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


## Data preprocessing

In [3]:
# Data imports
df1 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'QnGqQx-EA4g')
df2 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = '0jejyuRQLxc')
df3 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'gklQh5v0UuI')
df4 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'mcPzF-GSoiY')
df5 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'Q66BMCJMeHY')
df6 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = '-HLZwv_Eh7w')
df7 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'VwZFkMlUQGs')
df8 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'fCgbUyxXhAM')
df9 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'Fs1bel0HM6Y')
df10 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = '09PASKB3sgU')
df11 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'HfN1oPBGXRk')
df12 = pd.read_excel('Covid_vaccine_sentiments.xlsx',sheet_name = 'E3lSVH64u9k')

In [4]:
df1.shape[0] + df2.shape[0] + df3.shape[0] + df4.shape[0] + df5.shape[0] + df6.shape[0] + df7.shape[0] + df8.shape[0] + df9.shape[0] + df10.shape[0] + df11.shape[0] + df12.shape[0]       

2757

In [5]:
#Combining into one main data frame. # Comments without replies from all videos
frames_comments = [df1[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df2[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df3[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df4[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df5[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df6[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']], 
                        df7[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df8[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df9[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df10[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                        df11[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']],
                       df12[['Name', 'Comment', 'Time', 'Likes', 'Reply Count']]]
comments  = pd.concat(frames_comments).dropna().reset_index() 
comments['Comment'] = comments['Comment'].astype('str') # converting comments to strings


In [6]:
#Combining into one main data frame. # Comments without replies from all videos
frames_replies = [df1[['Reply Author','Reply', 'Published', 'Updated']],
                        df2[['Reply Author','Reply', 'Published', 'Updated']],
                        df3[['Reply Author','Reply', 'Published', 'Updated']],
                        df4[['Reply Author','Reply', 'Published', 'Updated']],
                        df5[['Reply Author','Reply', 'Published', 'Updated']],
                        df6[['Reply Author','Reply', 'Published', 'Updated']], 
                        df7[['Reply Author','Reply', 'Published', 'Updated']],
                        df8[['Reply Author','Reply', 'Published', 'Updated']],
                        df9[['Reply Author','Reply', 'Published', 'Updated']],
                        df10[['Reply Author','Reply', 'Published', 'Updated']],
                        df11[['Reply Author','Reply', 'Published', 'Updated']],
                        df12[['Reply Author','Reply', 'Published', 'Updated']]]
replies_to_comments = pd.concat(frames_replies).dropna().reset_index()
replies_to_comments['Reply'] = replies_to_comments['Reply'].astype('str') # Converting to string

In [19]:
replies_to_comments

,index,Reply Author,Reply,Published,Updated
0,52,Ruth Masango,Ramaphosa is not a stupid He knows whsts best...,2020-07-09T09:48:21Z,2020-07-09T09:48:21Z
1,57,Diane Wiese,The same trial is taking place in the UK and U...,2020-07-25T12:11:26Z,2020-07-25T12:11:26Z
2,58,martin mbokazi,@Diane WieseAt least i am talking rubbish. Yo...,2020-07-25T12:23:41Z,2020-07-25T12:23:41Z
3,59,Mroots1,@Diane Wiese are you certain of the vaccine tr...,2020-08-18T23:54:54Z,2020-08-18T23:54:54Z
4,60,Diane Wiese,Mroots1 how the hell would I know. All I know ...,2020-08-24T11:02:57Z,2020-08-24T11:02:57Z
...,...,...,...,...,...
1762,28,squeeze bag,It’s coming. Must be vaccinated in order to do...,2020-12-29T13:47:24Z,2020-12-29T13:47:24Z
1763,29,Ivile Harmans,@Chris Botha and when we forced to then what B...,2020-12-29T18:11:45Z,2020-12-29T18:11:45Z
1764,30,Ivile Harmans,@Sightseeing101 don&#39;t agree with that stat...,2020-12-29T18:12:12Z,2020-12-29T18:12:12Z
1765,31,Ivile Harmans,"@John Redman it&#39;s fine then, I&#39;m alrea...",2020-12-29T18:12:54Z,2020-12-29T18:12:54Z


In [7]:
#replies_to_comments.info()

In [8]:
# Removing url
replies_cleaned = [re.sub(r'^https?:\/\/.*[\r\n]*', '', replies_to_comments['Reply'].iloc[i], flags=re.MULTILINE) for i in range(len(replies_to_comments))]
comments_cleaned = [re.sub(r'^https?:\/\/.*[\r\n]*', '', comments['Comment'].iloc[i], flags=re.MULTILINE) for i in range(len(comments))]

In [9]:
#Removing html
replies_cleaned = [re.sub(r'<[^<]+?>', '', replies_cleaned[i], flags=re.MULTILINE) for i in range(len(replies_cleaned))]
comments_cleaned = [re.sub(r'<[^<]+?>', '', comments_cleaned[i], flags=re.MULTILINE) for i in range(len(comments_cleaned))]


In [10]:
# Removing special characters
replies_cleaned = [re.sub(r'[^A-Za-z0-9]+', ' ', replies_cleaned[i], flags=re.MULTILINE) for i in range(len(replies_cleaned))]
comments_cleaned = [re.sub(r'[^A-Za-z0-9]+', ' ', comments_cleaned[i], flags=re.MULTILINE) for i in range(len(comments_cleaned))]


In [18]:
replies_cleaned

['Ramaphosa is not a stupid He knows whsts best for Him',
 'The same trial is taking place in the UK and US You re talking rubbish',
 ' Diane WieseAt least i am talking rubbish You 39 re talking garbage because you cannot say for certainty what they are injecting to those South Africans',
 ' Diane Wiese are you certain of the vaccine trials being same quality quantity concentration and from same production material ',
 'Mroots1 how the hell would I know All I know is it s VOLUNTARY no one is being forced or coerced into volunteering some people will out of the goodness of their hearts I trust it s no different to trails overseas ',
 'They cause all kinds of problems down the road and I agree ',
 'START WITH MAADI AND HIS BRA PROF ABDROOOL GANGSTERS GREEDY KNIVING CROOKS',
 ' Those people have VOLUNTEERED No one is forcing them to do it If no one volunteered and they forced people that would be a problem but that s NOT the case All drugs have to be tested How did we get all the meds we 

In [11]:
#Tokenizing and making lower case
#reply_tokens = [re.findall("\w+", replies_cleaned[i].lower()) for i in range(len(replies_cleaned))]
#comment_tokens = [re.findall("\w+", comments_cleaned[i].lower()) for i in range(len(comments_cleaned))]
#Tokenizing but keeping cases
reply_tokens = [re.findall("\w+", replies_cleaned[i]) for i in range(len(replies_cleaned))]
comment_tokens = [re.findall("\w+", comments_cleaned[i]) for i in range(len(comments_cleaned))]


In [12]:
# Returning proper english words only
def going_english(sentences):
    """Input
       sentences: A list containing lists of tokenized sentences
       Output
       english_tokens: A list containing lists of tokenized sentences with english words only
    """
    english_tokens = []
    for i in range(len(sentences)):
        eng_wrds = []
        for word in sentences[i]:
            if word in english_words_set: # comparing with the imported english words
                eng_wrds.append(word)
        english_tokens.append(eng_wrds)
    return english_tokens

In [13]:
english_reply_tokens = going_english(reply_tokens)
english_comment_tokens = going_english(comment_tokens)

In [14]:
#english_comment_tokens

In [15]:
# Returning proper english words only
def remove_stopwords(sentences):
    """Input
       sentences: A list containing lists of tokenized sentences
       Output
       english_tokens: A list containing lists of tokenized sentences without english stopwords only
    """
    stop_words = get_stop_words('english')
    no_stopwords = []
    for i in range(len(sentences)):
        not_stopword = []
        for word in sentences[i]:
            if word not in stop_words: # comparing with the imported list of stopwords
                not_stopword.append(word)
        no_stopwords.append(not_stopword)
    return no_stopwords

In [16]:
cleaned_reply_tokens = remove_stopwords(english_reply_tokens)
cleaned_comment_tokens = remove_stopwords(english_comment_tokens)

In [47]:
# Returning proper english words only
def remove_single_letter_words(sentences):
    """Input
       sentences: A list containing lists of tokenized sentences
       Output
       english_tokens: A list containing lists of tokenized sentences without english stopwords only
    """
    single_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t'
                      ,'u','v','w','x','y','z']
    not_singles = []
    for i in range(len(sentences)):
        not_single = []
        for word in sentences[i]:
            if word not in single_letters: # comparing with the imported list of stopwords
                not_single.append(word)
        not_singles.append(not_single)
    return not_singles

In [48]:
cleaned_reply_tokens = remove_single_letter_words(cleaned_reply_tokens)
cleaned_comment_tokens = remove_single_letter_words(cleaned_comment_tokens)

In [17]:
cleaned_reply_tokens

[['stupid', 'best'],
 ['trial', 'place', 'UK', 're', 'rubbish'],
 ['Diane', 'least', 'rubbish', 're', 'garbage', 'say', 'certainty'],
 ['Diane', 'certain', 'vaccine', 'quality', 'quantity', 'material'],
 ['hell',
  'know',
  'know',
  's',
  'one',
  'people',
  'will',
  'trust',
  's',
  'different'],
 ['cause', 'road', 'agree'],
 [],
 ['people',
  'one',
  'one',
  'people',
  'problem',
  's',
  'case',
  'get',
  'world',
  'people'],
 ['Somali',
  'right',
  'congratulate',
  'Africa',
  'Africa',
  'conduct',
  'first',
  'although',
  'prefer',
  'independent',
  'run',
  'nevertheless',
  'Oxford',
  'Africa',
  'know',
  'will',
  'stay',
  'long',
  'haul',
  'even',
  'will',
  'long',
  'damage',
  'prevention',
  'way',
  'cure',
  'let',
  'alone',
  'ignore',
  'conspiracy',
  'theory',
  'accept'],
 ['look', 'can', 'find', 'lung', 'tissue', 'video'],
 ['s', 'straight', 'Gates', 'foundation'],
 ['use', 'famous', 'people', 'pandemic', 'real'],
 ['s', 'freedom', 'choice',

### Extra steps for data saving and and retrival because of package availability issues

In [ ]:
# I am missing some preprocessing packages on the kernel that has packages for training word embedding.
# so I first preprocess using another kernel, store the preprocessed tokenized word into data frames.
# I then switch to the kernel on which I'll be training the word embeddings . 

In [50]:
df_replies = pd.DataFrame(cleaned_reply_tokens)
df_comments = pd.DataFrame(cleaned_comment_tokens)

In [51]:
# Saving the preprocessed tokenized sentences to dataframes for use for other downstream tasks.
df_replies.to_csv('replies_dataframe.csv')
df_comments.to_csv('comments_dataframe.csv')


In [202]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# UNIT TEST COMMENT: Candidate for Table Driven Tests
# GRADED FUNCTION: get_count
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    
    word_count_dict = {}  # fill this with word counts
    ### START CODE HERE 
    word_count_dict.update(Counter(word_l))
            
    ### END CODE HERE ### 
    return word_count_dict

In [203]:
#Going over all sentences to get word counts and create word frequencies
# Depends on get_count function
def corpus_word_count(sentences):
    full_count_comments = {}
    for i in range(len(sentences)):
        full_count_comments.update(get_count(sentences[i]))
    return full_count_comments
#full_count

In [204]:
replies_word_counts = corpus_word_count(cleaned_reply_tokens)
comments_word_counts = corpus_word_count(cleaned_comment_tokens)

In [207]:
Vocub_replies = replies_word_counts.keys()
Vocub_comments = comments_word_counts.keys()

In [211]:
#Vocub_replies

In [167]:
 # Arranging in a descending order
comment_word_counts_sorted = sorted(full_count_comments.items(), key=lambda x: x[1], reverse=True)

In [168]:
comment_word_counts_sorted

[('politician', 5),
 ('pcr', 5),
 ('annoy', 5),
 ('quot', 4),
 ('group', 4),
 ('7', 4),
 ('feet', 4),
 ('attend', 4),
 ('hoof', 4),
 ('chew', 4),
 ('cud', 4),
 ('know', 3),
 ('39', 3),
 ('would', 3),
 ('dr', 3),
 ('bacteria', 3),
 ('my', 3),
 ('spread', 3),
 ('cell', 3),
 ('sa', 3),
 ('rate', 3),
 ('remedi', 3),
 ('homeless', 3),
 ('pass', 3),
 ('eat', 3),
 ('men', 3),
 ('plagu', 3),
 ('figur', 3),
 ('school', 3),
 ('tribul', 3),
 ('percept', 3),
 ('socialist', 3),
 ('0', 3),
 ('da', 3),
 ('curs', 3),
 ('among', 3),
 ('diagnos', 3),
 ('carl', 3),
 ('nerv', 3),
 ('biospher', 3),
 ('madagascan', 3),
 ('emit', 3),
 ('confirm', 3),
 ('rise', 3),
 ('adjuv', 3),
 ('unclean', 3),
 ('tsk', 3),
 ('him', 2),
 ('trial', 2),
 ('they', 2),
 ('from', 2),
 ('it', 2),
 ('s', 2),
 ('differ', 2),
 ('caus', 2),
 ('prof', 2),
 ('drug', 2),
 ('sister', 2),
 ('conduct', 2),
 ('oxford', 2),
 ('by', 2),
 ('freedom', 2),
 ('choic', 2),
 ('imbal', 2),
 ('make', 2),
 ('creator', 2),
 ('ha', 2),
 ('die', 2),
 ('s

In [212]:
 # Arranging in a descending order
reply_word_counts_sorted = sorted(replies_word_counts.items(), key=lambda x: x[1], reverse=True)

In [216]:
reply_word_counts_sorted[-20:]

[('description', 1),
 ('irrelevant', 1),
 ('formal', 1),
 ('poliomyelitis', 1),
 ('iron', 1),
 ('validate', 1),
 ('spare', 1),
 ('patch', 1),
 ('arrogant', 1),
 ('pastor', 1),
 ('vulgar', 1),
 ('guest', 1),
 ('cheek', 1),
 ('drank', 1),
 ('jess', 1),
 ('takeover', 1),
 ('smokescreen', 1),
 ('squeeze', 1),
 ('bag', 1),
 ('harsh', 1)]

In [ ]:
# Training Word Embeddings using gensim an NLP library
embedding  = word2vec.Word2Vec()

In [ ]:
# Stemming the words
ps = PorterStemmer() 
stemmed_reply_tokens = []
for i in range(len(reply_tokens)):
    sent = []
    for j in range(len(reply_tokens[i])):
        sent.append(ps.stem(reply_tokens[i][j]))
    stemmed_reply_tokens.append(sent)